In [1]:
# Imports
import numpy as np
import pickle
import scipy.stats as ss
import gurobipy as gp
from gurobipy import GRB

### Bin Packing

In [2]:
def DiscreteRand(mean, std, lb, ub):
    '''Sample from a discretized gaussian distribution truncated to [lb,ub].'''
    x = np.arange(lb, ub+1)
    xU, xL = x + 0.5, x - 0.5 
    prob = ss.norm.cdf(xU, loc=mean ,scale = std) - ss.norm.cdf(xL, loc=mean, scale = std)
    prob = prob / prob.sum() #normalize the probabilities so their sum is 1
    return np.random.choice(x, p = prob)

In [3]:
def PerfectBinPacking(mean, std, n, v):
    '''Create a perfect bin packing problem with piece size ~N(mean,std) with n bins of volume v'''
    items = {}
    for bins in range(n):
        b = v
        while b > 0:
            if b >= mean:
                item = DiscreteRand(mean,std,0,b)
            else:
                item = b
            b -= item
            if item in items.keys():
                items[item] += 1
            else:
                items.update({item : 1})    
    return items

In [4]:
def PackBins(items, volume, integer = False):
    '''A model for a bin packing problem.
    
    Args:
        items (Dict[int,int]): Dictionary from item size to number of those items
        volume (int): Volume of bins.
        integer (bool): Solve LP-relaxation if False; otherwise, solve IP.
    '''
    ITEMS = items.keys()
    BINS = range(sum(items.values())) # one item per bin
    v = volume
    count = items
    
    # define model
    m = gp.Model("pack_bins")
    
    # decision variables
    
    x = {} # the number of items of size i that are put in bin j
    for i in ITEMS:
        for j in BINS:
            if integer:
                x[i,j] = m.addVar(vtype=GRB.INTEGER, lb=0, ub=GRB.INFINITY, name=('(i%s, b%s)' % (i,j)))
            else:
                x[i,j] = m.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY, name=('(i%s, b%s)' % (i,j)))
                
    y = {} # 1 if bin i is used; 0 otherwise
    for i in BINS:
        if integer:
            y[i] = m.addVar(vtype=GRB.BINARY, name=('%s' % (i)))
        else:
            y[i] = m.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name=('%s' % (i)))
            
     # objective function
    m.setObjective(sum(y[i] for i in BINS), GRB.MINIMIZE)
    
    # constraints
    
    # subject to: bin volume
    for j in BINS:        
        m.addConstr(sum(x[i,j]*i for i in ITEMS) <= v, 'volume_%d' % (j))
        
    # subject to: all items packed
    for i in ITEMS:
        m.addConstr(sum(x[i,j] for j in BINS) == count[i], 'packed_%d' % (i))
        
    # subject to: bin used or not
    for j in BINS:
        M = v
        m.addConstr(sum(x[i,j]*i for i in ITEMS) <= M*y[j], 'used_%d' % (j))
    
    m.optimize()

In [7]:
#PerfectBinPacking(mean=35, std=5, n=50, v=150)
with open('data-bin_packing/items_168s.pickle', 'rb') as handle:
    items = pickle.load(handle)

In [8]:
PackBins(items, volume=150)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 525 rows, 10164 columns and 30008 nonzeros
Model fingerprint: 0xf2d49749
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve time: 0.02s
Presolved: 525 rows, 10164 columns, 30008 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 8.300e+01
 Factor NZ  : 5.320e+02
 Factor Ops : 1.052e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.27600327e+02  4.84000000e+01  5.60e+01 2.00e-15  5.88e-01     0s
   1   5.00000000e+01  4.85218626e+01  2.11e-15 3.75e-16  3.36e-02     0s
   2   5.00000000e+01  4.99985219e+01  9.33e-16 4.35e-16  3.36e-05     0s
   3  

In [9]:
PackBins(items, volume=150, integer=True)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 525 rows, 10164 columns and 30008 nonzeros
Model fingerprint: 0x222b03d1
Variable types: 0 continuous, 10164 integer (242 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 79.0000000
Presolve removed 242 rows and 0 columns
Presolve time: 0.04s
Presolved: 283 rows, 10164 columns, 20086 nonzeros
Variable types: 0 continuous, 10164 integer (2178 binary)

Root relaxation: objective 5.000000e+01, 710 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   50.00000    0   65   79.00000   50.00000  36.7%     -    0s
H    0     0                      55.0000000   50.00000  9.09%     -    0s
H    0     0                      54.000000